# 08b: Transform costs from literature

In this script, we transform the environmental external costs from literature sources (for electricity production) into common units.

In [20]:
%run common_definitions.py

In [21]:
import pandas as pd
import numpy as np

### Load data files

Sovacool et al:

In [22]:
df1 = pd.read_csv("../data/external_costs_sovacool.csv", delimiter="§")
df1 = df1[df1["Unit"] == "US¢/kWh"]
df1["Central Estimate ($2018)"] = df1["Central Estimate ($2018)"].astype(float)
df1["source name"] = "Sovacool et al"
df1["year"] = 2018
df1["cost"] = df1["Central Estimate ($2018)"] / 100
df1["unit"] = "kWh"
df1["currency"] = "dollar"
df1["index"] = df1.index

C:\Users\davidba\AppData\Local\Temp\ipykernel_13924\1014973640.py:1: ParserWarning: Falling back to the 'python' engine because the separator encoded in utf-8 is > 1 char long, and the 'c' engine does not support such separators; you can avoid this warning by specifying engine='python'.
  df1 = pd.read_csv("../data/external_costs_sovacool.csv", delimiter="§")


In [23]:
df1["Target"].unique()

array(['Coal', 'Gas', 'Oil', 'Nuclear', 'Wind', 'PV', 'Biomass Power',
       'Solar Thermal', 'Hydro', 'Biogas Power', 'Fossil Fuels',
       'Lignite', 'Waste Incineration', 'Peat', 'Geothermal', 'Fuel Cell',
       'All power generation', 'Coal and Nuclear'], dtype=object)

In [24]:
mapping_techs = {
    "Coal": "hard coal power plant",
    "Gas": "natural gas power plant",
    "Oil": "oil power plant",
    "Nuclear": "nuclear, pressure water reactor",
    "Wind": "wind, onshore, medium",
    "PV": "PV commercial",
    "Biomass Power": "wood CHP modern",
    "Solar Thermal": "CSP, tower",
    "Hydro": "hydro, reservoir, alpine region",
    "Lignite": "lignite power plant",
    "Geothermal": "geothermal"
}

In [25]:
df1 = df1[df1["Target"].isin(list(mapping_techs.keys()))]
df1["short name"] = df1["Target"].apply(lambda x: mapping_techs[x])

NEEDS project. We select the 2025 technology year.

In [26]:
df2 = pd.read_csv("../data/external_costs_NEEDS.csv").fillna(0)
df2 = df2[df2["technology year"] == 2025]

Create four different summations with the climate change impacts.

In [27]:
cc_variants = ["damage costs low", "damage costs high", "abatement costs low", "abatement costs high"]
other_ics = ["health impacts", "biodiversity", "crop yield losses", "material damage", "land use"]

In [28]:
df2_pivot = df2.pivot(columns="impact category", values="cost", index="short name")

In [29]:
data = {}
for v in cc_variants:
    data[v] = df2_pivot[other_ics+["climate change, "+v]].sum(axis=1).to_numpy()

df2_new = pd.DataFrame(
    data,
    index = df2_pivot.index
).melt(var_name="index", value_name="cost", ignore_index=False).reset_index()

In [30]:
df2_new["unit"] = "kWh"
df2_new["currency"] = "euro"
df2_new["year"] = 2000
df2_new["source name"] = "NEEDS"

In [31]:
df3 = pd.read_csv("../data/external_costs_trinomics.csv")
df3["index"] = 0

### Combine all sources

In [32]:
df = pd.concat(
    (
        df1[["source name", "short name", "index", "unit", "currency", "year", "cost"]],
        df2_new[["source name", "short name", "index", "unit", "currency", "year", "cost"]],
        df3[["source name", "short name", "index", "unit", "currency", "year", "cost"]]
    ),
    axis=0,
)

### Transform currency units

Here, we first adjust for inflation, then convert to euros.

In [33]:
HICP = pd.read_csv("../data/HICPdata.csv").set_index("year")
HICP = HICP["average HICP"]
PPP = pd.read_csv("../data/PPPdata.csv").set_index("year")["PPP"]
CPI = pd.read_csv("../data/CPI_US.csv").set_index("year")["CPI"]

def convert_to_euros(currency, year, target_year):
    if currency == "euro":
        return HICP[target_year] / HICP[year]
    if currency == "dollar":
        if year in PPP.index:
            # adjust for inflation, then convert currency
            return PPP[target_year] * (CPI[target_year] / CPI[year])
        else:
            # convert to last year in PPP data
            last_year = max(PPP.index)
            return (HICP[target_year] / HICP[last_year]) * PPP[last_year] * (CPI[last_year]/CPI[year])

In [34]:
unit2MJ = {
    "MWh": 3600,
    "GJ": 1e03,
    "MJ": 1.0,
    "kWh": 3.6
}

In [35]:
df["conversion factor"] = (df.apply(lambda x: convert_to_euros(x.currency, x.year, EURO_REF_YEAR), axis=1) 
                           / df["unit"].map(unit2MJ))
df["cost"] = df["cost"] * df["conversion factor"]
df["currency"] = "euro"
df["year"] = EURO_REF_YEAR
df["unit"] = "MJ"

In [36]:
df.drop("conversion factor", axis=1, inplace=True)
df.to_csv("../data/external_costs_literature_EUR{}.csv".format(EURO_REF_YEAR), index=False)